# Environment Setting Up

In [1]:
import os
from dotenv import load_dotenv

# Loading environment variables from .env
load_dotenv()

# Changing directory to main directory for easy data access
working_directory = os.getenv("WORKING_DIRECTORY")
os.chdir(working_directory)

# Checking the change
%pwd

'/workspaces/Live-Air-Quality'

In [2]:
from pathlib import Path

# Checking the change
print("Git folder exists:", Path(".git").exists())

Git folder exists: True


# 6. Dashboard

In [3]:
import plotly.express as px
import duckdb as ddb
import pandas as pd

from dash import Dash, dcc, html, Input, Output

database_path = Path("artifacts/sql/air_quality.db")

with ddb.connect(database_path, read_only=True) as conn:
    data = conn.execute("SELECT * FROM presentation.air_quality_data;").fetchdf()
    data_lastest_values = conn.execute("SELECT * FROM presentation.latest_params_per_location;").fetchdf()
    data_daily_stats = conn.execute("SELECT * FROM presentation.daily_stats;").fetchdf()

print(f"All Data: {data.shape}")
print(f"Latest Params: {data_lastest_values.shape}")
print(f"Daily Stats: {data_daily_stats.shape}")

All Data: (16807, 11)
Latest Params: (24, 7)
Daily Stats: (775, 11)


In [4]:
data_lastest_values.head(2)

,location_id,location,lat,lon,datetime,pm25,o3
0,971,Elizabeth Trailer-971,40.641441,-74.208366,2025-02-01 05:00:00,14.4,NaN
1,665,Bronx - IS74-665,40.814701,-73.886703,2025-02-01 05:00:00,15.8,NaN


In [5]:
data_daily_stats.head(2)

,location_id,location,measurement_date,weekday_number,weekday,is_weekend,lat,lon,parameter,units,average_value
0,384,CCNY-384,2025-01-01,3,Wednesday,0,40.8197,-73.9481,o3,ppm,0.01650
1,384,CCNY-384,2025-01-03,5,Friday,0,40.8197,-73.9481,o3,ppm,0.01819


In [18]:
# Scatter plot of all the AQ locations and their latest values

def location_scatter_map():
    location_scatter = px.scatter_map(
        data_lastest_values,
        lat="lat",
        lon="lon",
        hover_name="location",
        hover_data={"lat": False, "lon": False, "datetime": True, "pm25": True, "o3": True},
        zoom=10.0,
    )

    location_scatter.update_layout(
        mapbox_style="open-street-map",
        height=800,
        title="AQ Monitoring Locations"
        )
    
    return location_scatter

In [15]:
def line_figure():
    line_fig = px.line(
        data_daily_stats[data_daily_stats["parameter"] == "pm25"].sort_values(by="measurement_date"),
        x="measurement_date",
        y="average_value",
        title="Plot over time of PM25 levels."
    )

    return line_fig

In [16]:
def box_figure():
    box_fig = px.box(
        data_daily_stats[data_daily_stats["parameter"] == "pm25"].sort_values(by="weekday_number"),
        x="weekday",
        y="average_value",
        title="Distribution of PM25 levels by Weekday"
    )

    return box_fig

In [19]:
app = Dash(__name__)
app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label="Sensor Locations", children=[dcc.Graph(id="Map-View", figure=location_scatter_map())]),
        dcc.Tab(label="Parameter Plots", children=[
            dcc.Graph(id="Line-Plot", figure=line_figure()),
            dcc.Graph(id="Box-Plot", figure=box_figure())
        ])
    ])
])

if __name__ == "__main__":
    app.run(jupyter_mode="external", debug=True)

Dash app running on http://127.0.0.1:8050/
